In [2]:
import duckdb
import pandas as pd

# import plotly_express as px

%load_ext magic_duckdb

In [28]:
%%dql
FORCE INSTALL delta FROM core_nightly;

,Success


In [31]:
%%dql
UPDATE EXTENSIONS

,extension_name,repository,update_result,previous_version,current_version
0,azure,core,NO_UPDATE_AVAILABLE,b0ffe7a,b0ffe7a
1,httpfs,core,NO_UPDATE_AVAILABLE,v1.1.1,v1.1.1
2,delta,core_nightly,NO_UPDATE_AVAILABLE,v0.2.1,v0.2.1


In [54]:
%%dql
CREATE OR REPLACE SECRET cheese (TYPE AZURE, PROVIDER CREDENTIAL_CHAIN, SCOPE 'abfss://lrndlkcheese12345hngc.dfs.core.windows.net/')

,Success
0,True


In [33]:
%%dql
SET azure_transport_option_type = 'curl'; --important for when running in linux!
SET azure_http_stats = false;
SET azure_read_transfer_concurrency = 5;
SET azure_read_transfer_chunk_size = 1_048_576;
SET azure_read_buffer_size = 1_048_576;

,Success


In [6]:
%%dql
DESCRIBE
FROM 'abfss://lrndlkcheese12345hngc.dfs.core.windows.net/lake/csv/cheeses.csv';

,column_name,column_type,null,key,default,extra
0,cheese,VARCHAR,YES,None,None,None
1,url,VARCHAR,YES,None,None,None
2,milk,VARCHAR,YES,None,None,None
3,country,VARCHAR,YES,None,None,None
4,region,VARCHAR,YES,None,None,None
5,family,VARCHAR,YES,None,None,None
6,type,VARCHAR,YES,None,None,None
7,fat_content,VARCHAR,YES,None,None,None
8,calcium_content,VARCHAR,YES,None,None,None
9,texture,VARCHAR,YES,None,None,None


In [55]:
%%dql
SELECT *
FROM 'abfss://lrndlkcheese12345hngc.dfs.core.windows.net/lake/csv/cheeses.csv'
LIMIT 5;

,cheese,url,milk,country,region,family,type,fat_content,calcium_content,texture,rind,color,flavor,aroma,vegetarian,vegan,synonyms,alt_spellings,producers
0,Aarewasser,https://www.cheese.com/aarewasser/,cow,Switzerland,NA,NA,semi-soft,NA,NA,buttery,washed,yellow,sweet,buttery,FALSE,FALSE,NA,NA,Jumi
1,Abbaye de Belloc,https://www.cheese.com/abbaye-de-belloc/,sheep,France,Pays Basque,NA,"semi-hard, artisan",NA,NA,"creamy, dense, firm",natural,yellow,burnt caramel,lanoline,TRUE,FALSE,Abbaye Notre-Dame de Belloc,NA,NA
2,Abbaye de Belval,https://www.cheese.com/abbaye-de-belval/,cow,France,NA,NA,semi-hard,40-46%,NA,elastic,washed,ivory,NA,aromatic,FALSE,FALSE,NA,NA,NA
3,Abbaye de Citeaux,https://www.cheese.com/abbaye-de-citeaux/,cow,France,Burgundy,NA,"semi-soft, artisan, brined",NA,NA,"creamy, dense, smooth",washed,white,"acidic, milky, smooth","barnyardy, earthy",FALSE,FALSE,NA,NA,NA
4,Abbaye de Tamié,https://www.cheese.com/tamie/,cow,France,Savoie,NA,"soft, artisan",NA,NA,"creamy, open, smooth",washed,white,"fruity, nutty","perfumed, pungent",FALSE,FALSE,NA,"Tamié, Trappiste de Tamie, Abbey of Tamie",NA


In [50]:
%%dql
CREATE OR REPLACE SECRET delta (
    TYPE AZURE,
    PROVIDER CREDENTIAL_CHAIN,
    CHAIN 'cli;default',
    ACCOUNT_NAME 'lrndlkcheese12345hngc'
);

,Success
0,True


In [58]:
%%dql
SELECT *
FROM delta_scan('abfss://lake/csv/cheeses/')
LIMIT 5;

ValueError: Error executing SELECT *
FROM delta_scan('abfss://lake/csv/cheeses/')
LIMIT 5; in DuckDB

In [3]:
%%time
%%dql -t show
SELECT * FROM 'https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet' LIMIT 5;

┌───────────┬─────────────────────┬─────────────────────┬─────────────────┬───────────────┬──────────────┬────────────────────┬────────────────────┬─────────────────────┬──────────────┬─────────────┬───────┬─────────┬────────────┬──────────────┬───────────────────────┬──────────────┬──────────────────────┐
│ vendor_id │      pickup_at      │     dropoff_at      │ passenger_count │ trip_distance │ rate_code_id │ store_and_fwd_flag │ pickup_location_id │ dropoff_location_id │ payment_type │ fare_amount │ extra │ mta_tax │ tip_amount │ tolls_amount │ improvement_surcharge │ total_amount │ congestion_surcharge │
│  varchar  │      timestamp      │      timestamp      │      int8       │     float     │   varchar    │      varchar       │       int32        │        int32        │   varchar    │    float    │ float │  float  │   float    │    float     │         float         │    float     │        float         │
├───────────┼─────────────────────┼─────────────────────┼─────────────────┼─

In [ ]:
%%time
%%dql -t show
SELECT * FROM 'https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet' LIMIT 5;

In [1]:
%%dql -t df
SELECT 
       format('{:,}', CAST(ROUND((sum(total_compressed_size) / (1024))) AS INT)) as compressed_KB,
       format('{:,}', CAST(ROUND((sum(total_uncompressed_size) / (1024))) AS INT)) as uncompressed_KB,
       format('{:,}', CAST(ROUND((sum(total_compressed_size) / (1024 * 1024))) AS INT)) as compressed_MB,
       format('{:,}', CAST(ROUND((sum(total_uncompressed_size) / (1024 * 1024))) AS INT)) as uncompressed_MB,
       sum(total_compressed_size) / (1024 * 1024 * 1024) as compressed_GB,
       sum(total_uncompressed_size) / (1024 * 1024 * 1024) as uncompressed_GB    
FROM parquet_metadata('https://github.com/cwida/duckdb-data/releases/download/v1.0/taxi_2019_04.parquet')

UsageError: Cell magic `%%dql` not found.
